<a href="https://colab.research.google.com/github/ketkiambekar/NJIT-Capstone-Project/blob/main/Topic%20Classification/EAFC_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
text="I received my result yesterday and my cancer is in remission, yay!!"

In [87]:
pip install contractions

In [88]:
#from sklearn.svm import LinearSVC
import joblib
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords, wordnet
import string
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
#import text_laundry


In [89]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [90]:
def remove_punctuations(text):
    punc = string.punctuation 
    for punctuation in punc:
        text = text.replace(punctuation, '')
    return text

In [91]:
def remove_contractions(text):
  return " ".join([contractions.fix(word) for word in text.split()])

In [92]:
def remove_stopwords(text, filename):
  stop_words=[]
  with open(filename,'r') as f:
    for line in f:
      for word in line.split():
        stop_words.append(word.lower()) 
  result =  [word for word in text if word not in stop_words]
  return result

In [93]:

# text="I am in remission yay"
# tokens= word_tokenize(text)
# print(tokens)
# tokens = remove_stopwords(tokens, 'stopwords.txt')

# print(tokens)

In [94]:
def replace_strings(text, new_string , remove_arr ):
  for r in remove_arr:
    text = text.replace(r, new_string)
  return text

In [95]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [96]:
# #Tag Wordnet position
# wn_pos = [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
# print(wn_pos)
# print(type(wn_pos))
# wnl = WordNetLemmatizer()
# lemmatized=" ".join([wnl.lemmatize(word, tag) for word, tag in wn_pos])
# print(lemmatized)
# print(type(lemmatized))


In [97]:
def Lemmatize(text):
  #Make Lower Case
  text=text.lower()

  #Remove Word contractions
  text = remove_contractions(text)

  #Remove Punctuation
  text = remove_punctuations(text)


  #Replace Strings

  #Replace all days of week by string "dayofweek"
  weekstring =['monday', 'tuesday', 'wednesday', 'thursday', 'friday','saturday','sunday']
  text = replace_strings(text,'dayofweek', weekstring)

  #Replace all month names by string "month"
  months = ['january','february', 'march','april','may','june','july','august', 'september', 'october','november','december']
  text = replace_strings(text,'month', months)

  #Replace all Numbers by String 'number'
  nums=['one','two','three','four','five','six','seven','eight','nine','ten']
  text = replace_strings(text, 'number',nums)

  #Replace all family relations with string "famrel"
  famrel=['dad','mum','daughter','son','aunt','uncle','husband','wife', 'brother','sister', 'father','mother']
  text = replace_strings(text, 'famrel',famrel)
  
  #Replace all friendship (non-family) relations with string "friend"
  friendrel=['boyfriend','girlfriend','colleague'] 
  text = replace_strings(text, 'friend',friendrel)

  #Tokenize
  tokens= word_tokenize(text)

  #Remove StopWords
  tokens = remove_stopwords(tokens, 'stopwords.txt')
  #print(tokens)

  #Tagging Parts of Speech in the tokenized 
  pos = nltk.tag.pos_tag(tokens)
  #print(pos)

  #Tag Wordnet position
  wn_pos = [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]

  #Get Lemmatized String
  wnl = WordNetLemmatizer()
  lemmatized=" ".join([wnl.lemmatize(word, tag) for word, tag in wn_pos])

  return lemmatized

In [98]:
def Vectorize(text, vectorizer):
  #n-gram Tokenization
  print(text)
  vectorized =  vectorizer.transform([text])
  print(vectorized)
  return vectorized

  

In [99]:
vectorizer=joblib.load("EAFC_vectorizer.joblib")

In [100]:
model = joblib.load("EAFC_TC.joblib")

In [107]:
text = Lemmatize(text)

In [108]:
print(text)

receive result yesterday cancer remission yay


In [109]:
text_vectorized = Vectorize(text, vectorizer)


receive result yesterday cancer remission yay
  (0, 7965)	0.38236320007347296
  (0, 7911)	0.6227682700968598
  (0, 5936)	0.34476600151730785
  (0, 5904)	0.4017019338124211
  (0, 5850)	0.4112439899413921
  (0, 1102)	0.12887360589488803


In [110]:
y_pred=model.predict(text_vectorized)
if y_pred[0]==0:
  result = "Class 0: Newly Discovered"
elif y_pred[0]==1:
  result = "Class 1: Venting/Sharing"
elif y_pred[0]==2:
  result = "Class 2: Reaching out for help"
elif y_pred[0]==3:
  result = "Class 3: Passing Away"
elif y_pred[0]==4:
  result = "Class 4: Remission"

In [111]:
print(result)

Class 4: Remission
